In [2]:
# | default_exp agent.debater

In [1]:
# | export
from enum import Enum
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import asyncio
from agentic.core.agent import Agent, AgentConfig
from agentic.llms.client import LLMClient


In [2]:
# | export
class DebateRole(Enum):
    ADVOCATE = "advocate"
    CRITIC = "critic"
    EXPERT = "expert"
    MODERATOR = "moderator"

@dataclass
class DebateConfig:
    topic: str
    context: str
    max_rounds: int = 2
    time_limit: Optional[int] = None

class DebateAgent(Agent):
    """Multi-agent debate system for structured decision making"""

    def __init__(self, config: AgentConfig, role: DebateRole, debate_config: DebateConfig):
        super().__init__(config)
        self.role = role
        self.debate_config = debate_config
        self.debate_history: List[Dict[str, str]] = []

        self.role_prompts = {
            DebateRole.ADVOCATE: f"You are advocating FOR the position: {debate_config.topic}. Present strong arguments with evidence.",
            DebateRole.CRITIC: f"You are arguing AGAINST the position: {debate_config.topic}. Challenge with counterarguments and evidence.",
            DebateRole.EXPERT: f"You are a domain expert analyzing: {debate_config.topic}. Provide balanced technical insights.",
            DebateRole.MODERATOR: f"You moderate the debate on: {debate_config.topic}. Synthesize arguments and provide final verdict."
        }

    async def opening_statement(self) -> str:
        """Generate opening statement based on role"""
        prompt = f"""
{self.role_prompts[self.role]}

Context: {self.debate_config.context}

Provide your opening statement:
- State your main position clearly
- Provide 2-3 key arguments supporting your stance
- Preview technical evidence or examples to back your claims
"""
        messages = [{"role": "user", "content": prompt}]
        response = self.llm_client.create_completion(messages=messages, stream=True)
        result = self.llm_client.handle_streaming_response(response)

        text = result.get("content", "") if isinstance(result, dict) else str(result)
        self.debate_history.append({"role": self.role.value, "type": "opening", "content": text})
        return text

    async def respond_to_debate(self, previous_statements: List[Dict]) -> str:
        """Respond to previous debate statements with context"""
        # Format previous statements for context
        debate_context = "\n\n".join(
            [f"{stmt['role'].capitalize()} ({stmt['type']}): {stmt['content']}" 
             for stmt in previous_statements]
        )

        prompt = f"""
{self.role_prompts[self.role]}

Context: {self.debate_config.context}

Previous debate statements:
{debate_context}

Respond to the ongoing debate (250-300 words):
- Address specific points raised by other agents
- Reinforce or refute arguments with technical reasoning
- Provide concise, evidence-based arguments
"""
        messages = [{"role": "user", "content": prompt}]
        response = self.llm_client.create_completion(messages=messages, stream=True)
        result = self.llm_client.handle_streaming_response(response)

        text = result.get("content", "") if isinstance(result, dict) else str(result)
        self.debate_history.append({"role": self.role.value, "type": "response", "content": text})
        return text

    async def final_verdict(self, all_statements: List[Dict]) -> str:
        """Generate final verdict (moderator only)"""
        if self.role != DebateRole.MODERATOR:
            return ""

        context = "\n\n".join(
            [f"{stmt['role'].capitalize()} ({stmt['type']}): {stmt['content']}" 
             for stmt in all_statements]
        )

        prompt = f"""
{self.role_prompts[self.role]}

Context: {self.debate_config.context}

All debate statements:
{context}

Provide the final verdict (200-250 words):
1. Summarize key arguments from each side
2. Identify the strongest points made
3. Provide a final recommendation with clear reasoning
4. Discuss implementation considerations
"""
        messages = [{"role": "user", "content": prompt}]
        response = self.llm_client.create_completion(messages=messages, stream=True)
        result = self.llm_client.handle_streaming_response(response)

        text = result.get("content", "") if isinstance(result, dict) else str(result)
        self.debate_history.append({"role": self.role.value, "type": "verdict", "content": text})
        return text

async def create_debate(topic: str, context: str, max_rounds: int = 2) -> Dict[str, Any]:
    """Create and run a structured debate"""
    debate_config = DebateConfig(topic=topic, context=context, max_rounds=max_rounds)

    base_config = AgentConfig(name="debate_agent", model="qwen3:8b")
    agents = {
        DebateRole.ADVOCATE: DebateAgent(base_config, DebateRole.ADVOCATE, debate_config),
        DebateRole.CRITIC: DebateAgent(base_config, DebateRole.CRITIC, debate_config),
        DebateRole.EXPERT: DebateAgent(base_config, DebateRole.EXPERT, debate_config),
        DebateRole.MODERATOR: DebateAgent(base_config, DebateRole.MODERATOR, debate_config),
    }

    all_statements: List[Dict] = []

    # Opening statements
    for role in [DebateRole.ADVOCATE, DebateRole.CRITIC, DebateRole.EXPERT]:
        statement = await agents[role].opening_statement()
        all_statements.append({"role": role.value, "type": "opening", "content": statement})

    # Discussion rounds
    for round_num in range(max_rounds):
        for role in [DebateRole.ADVOCATE, DebateRole.CRITIC, DebateRole.EXPERT]:
            response = await agents[role].respond_to_debate(all_statements)
            all_statements.append({"role": role.value, "type": f"round_{round_num+1}", "content": response})

    # Final verdict
    verdict = await agents[DebateRole.MODERATOR].final_verdict(all_statements)
    return verdict
    # return {
    #     "topic": topic,
    #     "context": context,
    #     "statements": all_statements,
    #     "verdict": verdict,
    #     "rounds": max_rounds,
    # }

In [3]:
await create_debate(
    topic="Kya Shaktimaan hi Gangadhar hai…? Ya fir Gangadhar sirf ek bechara photographer hai aur hum sab ek bade dhokhe mein jee rahe hain?",
    context=(
        "Aaj hum uthayenge ek aisa sawaal jo har asli 90s bachpan ka hissa raha hai… ek raaz, ek pehchaan, ek chakravyuh jisme poori generation atki hui hai: "
        "Kya Gangadhar — woh naazuk sa, hamesha girta padta photographer — asal mein Bharat ka pehla aur asli superhero, Shaktimaan hai? "
        "Ya fir yeh sab ek dhoka hai? Ek aisi chhalava jisme ek chashma poore desh ko bewakoof bana gaya? "
        "Akhir kitne coincidences honge? Kitni baar Gangadhar tabhi kyun gayab hota hai jab Shaktimaan entry leta hai? "
        "Debaters ko in spine-chilling, dimaag-hila dene waale clues par roshni daalni hogi: "
        "1. Dono ka chehra — ekdum photocopy! Matlab Xerox machine bhi sharma jaye, "
        "2. Woh chashma — kya ye sirf power glasses hain ya invisibility ka jaadu hai? Harry Potter bhi confuse ho jaye, "
        "3. Gangadhar ka timing — danger aate hi gayab, Shaktimaan aate hi active. Kuch toh gadbad hai Daya! "
        "4. Kya koi insaan itni tezi se ghoom ke hawa mein udd sakta hai bina chakravyuh mein fase? NASA bhi soche, ‘How bro?’ "
        "5. Aur Tun-Tun Aunty… kya unhone sach jaan liya tha? Aur kya unki loyalty sirf free samoson ke liye bik gayi thi? "
        "Taiyaar ho jaaiye ek mahaan sach se parda uthane ke liye. Yeh sirf debate nahi — yeh hai ek quest for sachchai!"
        "Kripya Hinglish mein discuss karein."
    ),
    max_rounds=1,
)

2025-10-12 00:03:18,857 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:03:18,870 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:03:18,883 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:03:18,895 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:03:29,235 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, so the user wants me to advocate for the position that Gangadhar is actually Shaktimaan, not just a photographer. They mentioned using strong arguments with evidence. Let me start by recalling the context.

First, the main question is whether Gangadhar is the real superhero or if it's all a scam. The user provided several clues like the identical faces, the mysterious glasses, timing issues, and Tun-Tun Aunty's loyalty. I need to structure the opening statement clearly.

The user specified to use Hinglish, so I should mix Hindi and English as needed. My main position is that Gangadhar is Shaktimaan. The key arguments should be based on the evidence given.

First argument: The faces being exact copies. If Gangadhar's face is a photocopy of Shaktimaan, maybe it's the same person using different identities. The user mentioned Xerox machine being embarrassed, which emphasizes the similarity.

Second argument: The glasse

Opening Statement:

Aaj hum log uthayenge ek raaz jisme har 90s bachpan wala uthna pade ga: Kya Gangadhar sirf ek bechara摄影师 hai ya 
woh Bharat ka asli superhero Shaktimaan hai? Mai apni tarf se itna kahun ge ke Gangadhar itna aadmi hai jo         
Shaktimaan hai, aur yeh sirf ek chashma ke behind ka game hai. Mai aapko 3 kriya-pramukh argument dede ga jo yeh   
sab sath saath uthade ga.

1. चेहरा-एक-दम-फोटोकॉपी! (Face - Photocopy!)

...

The Critic and Advocate counter that the identical faces, glasses, and timing are mere technical limitations or    
narrative gimmicks, suggesting a hidden agenda. While valid, these arguments overlook the show’s self-aware design,
which explicitly embraced ambiguity as a storytelling tool. The Expert’s analysis of real-world parallels (e.g.,   
adaptive optics) and the show’s revival-era acknowledgment of the "trick" as playful symbolism strengthens this    
view.

Strongest Points: The Expert’s metafictional lens and Advocates’ emphasis on creative storytelling prevail. The    
narrative’s truth lies in its ability to provoke questions about reality, not in hidden secrets.

Recommendation: Recognize the duality as a deliberate narrative strategy, not deception. This approach honors the  
show’s artistic intent and invites audiences to engage with its layers of symbolism and imagination.

Implementation Considerations: Present the debate’s conclusion by highlighting the show’s blend of creativity,     
technology, and metafiction, ensuring audience understanding of its symbolic layers while avoiding simplistic      
"dhokha" interpretations. This framing celebrates storytelling’s power to shape perception, aligning with the      
Expert’s vision of narrative illusion as a form of creative genius. 🌟

'\n\n**Final Verdict:**  \n\nThe debate centers on whether the "Gangadhar-Shaktimaan" duality is a narrative strategy or a deliberate deception. The **Expert** and **Advocate** argue it is a masterclass in metafiction, using symbolic duality, layered storytelling, and tech speculation to engage audiences. They emphasize the show’s intentional use of camera tricks, multifunctional symbolism (e.g., glasses as AR devices), and meta-commentary on audience complicity via Tun-Tun Aunty. These points highlight creativity and narrative depth, framing the mystery as a celebration of imagination rather than a "dhokha."  \n\nThe **Critic** and **Advocate** counter that the identical faces, glasses, and timing are mere technical limitations or narrative gimmicks, suggesting a hidden agenda. While valid, these arguments overlook the show’s self-aware design, which explicitly embraced ambiguity as a storytelling tool. The **Expert**’s analysis of real-world parallels (e.g., adaptive optics) and the 

In [4]:
await create_debate(
    topic="Is Shaktimaan actually Gangadhar in disguise, or are we all victims of the greatest glasses-based illusion in Indian history?",
    context=(
        "Let’s settle the most important question that haunted every 90s kid more than missing an episode of Shaktimaan after homework: "
        "Is Gangadhar, the awkward photographer with the fashion sense of a confused librarian, secretly the nation's first chakra-spinning superhero? "
        "Or is it possible that two people with the exact same face, voice, and suspicious schedule just happen to exist in the same city, and no one questioned it because... 90s logic? "
        "Debaters must bravely tackle the following clues without laughing (too hard): "
        "1. The fact that Gangadhar and Shaktimaan look more alike than cloned potatoes, "
        "2. The mysterious power of his chashma, which might be less 'optical aid' and more 'Harry Potter relic,' "
        "3. Gangadhar vanishes faster than your salary after Zomato’s Great Indian Food Fest whenever danger arises, "
        "4. The scientific possibility of a human spinning into the stratosphere without getting vertigo, "
        "5. And whether Tun-Tun Aunty has known the truth all along but kept quiet because Shaktimaan promised her unlimited samosas and zero rent."
    ),
    max_rounds=1,
)


2025-10-12 00:05:31,679 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:05:31,693 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:05:31,705 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:05:31,718 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:05:31,842 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, so I need to argue that Shaktimaan is actually Gangadhar in disguise, right? Let me start by recalling the context. The user wants strong arguments with evidence, and they mentioned specific clues like the appearance, the chashma's power, Gangadhar's vanishing act, the spinning without vertigo, and Tun-Tun Aunty's possible knowledge.

First, the main position: Yes, Shaktimaan is Gangadhar. Now, I need to back this up. Let me think about each clue. 

1. The first clue is their identical looks. If they look exactly the same, that's suspicious. Maybe it's a disguise. But how? Maybe Gangadhar uses some kind of tech or magical item to change his appearance. The chashma (glasses) is mentioned as a key point. The glasses might not just be for vision but have some supernatural power, maybe like Harry Potter's glasses. That could be part of his disguise, allowing him to change appearance or see things others can't. 

2. Gang

Opening Statement:

The evidence overwhelmingly suggests that Shaktimaan is indeed Gangadhar in disguise, not a separate entity but a  
dual identity orchestrated by the show’s creators to craft a mythological-adjacent narrative. This theory is       
supported by the uncanny physical and behavioral parallels between the two, the mystical properties of Gangadhar’s 
chashma (glasses), and the deliberate narrative choices that hint at a deeper, almost supernatural truth. Let’s    
dismantle the “90s logic” and explore the clues that point to a single, extraordinary individual masquerading as   
two.

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

 • Emphasize the cultural and historical context of the show to frame it as a mythic legend, not a puzzle to solve.

 • Highlight the symbolic role of elements like the chashma to align with the show’s thematic depth.

 • Acknowledge the show’s reliance on fictional logic to maintain its nostalgic appeal, avoiding rigid             
   interpretations that clash with its artistic intent.

Ultimately, the show’s enduring magic lies in its ability to invite interpretation, making it a testament to the   
power of storytelling rather than a mystery to solve.

'\n\n**Final Verdict (200-250 words):**  \nThe debate over whether Gangadhar and Shaktimaan are one person or distinct characters hinges on interpreting the show’s symbolic, narrative, and cultural elements. The **Advocate** emphasizes physical resemblance, the chashma’s mystical role, and Tun-Tun Aunty’s complicity as clues to a unified identity, framing the show as a mythic blend of divine power and storytelling. The **Critic and Expert**, however, argue that these elements are narrative tools designed to evoke mystery and cultural archetypes, not literal proof of a hidden identity. They highlight the show’s 1990s fictional premise, which thrives on blending myth and reality, rather than scientific plausibility.  \n\n**Strongest Points:**  \n- The Advocate’s cultural context and symbolic interpretation (e.g., the chashma as a divine symbol) offer a compelling lens to view the show as a metaphor for duality.  \n- The Critic and Expert’s focus on narrative devices and the show’s fictio

In [5]:
# Example Usage 
# config = DebateConfig()
await create_debate(
    topic="Which framework/language/technique is best for building a Kubernetes operator to monitor resource health?",
    context=(
        "The task is to build a Kubernetes operator that continuously monitors health of core resources such as Pods, "
        "Services, Deployments, and DaemonSets. Several approaches are possible: "
        "using Go with the Operator SDK (closer to Kubernetes ecosystem), "
        "using Python with Kopf (faster prototyping and developer-friendly), "
        "or leveraging Rust (performance and safety). "
        "The debate should consider not only technical fit, but also broader criteria such as: "
        "1. Open-source maturity and adoption within the Kubernetes community, "
        "2. Long-term security support and patch availability, "
        "3. Frequency of stable releases and active maintenance, "
        "4. Ease of onboarding and developer productivity, "
        "5. Integration with existing monitoring and observability tools, "
        "6. Performance and scalability under large workloads."
    ),
    max_rounds=1,
)


2025-10-12 00:07:50,128 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:07:50,143 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:07:50,155 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:07:50,168 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:07:50,272 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user is asking which framework/language/technique is best for building a Kubernetes operator to monitor resource health. The options are Go with Operator SDK, Python with Kopf, or Rust. I need to advocate for one of them. Let me start by understanding the requirements.

First, the main task is to monitor core Kubernetes resources like Pods, Services, etc. The user mentioned several criteria: open-source maturity, security, release frequency, ease of onboarding, integration with observability tools, and performance. 

I should consider each of these points. Let's break down each option. 

Go with Operator SDK is deeply integrated with Kubernetes. The Operator SDK is a CNCF project, which means it's well-maintained and has strong community support. That's a big plus for open-source maturity. Also, Go is a systems language, so it's efficient and can handle performance requirements. Operators are typically written i

Opening Statement:

The optimal framework for building a Kubernetes operator to monitor resource health is Go with the Operator SDK.   
This choice balances technical depth, ecosystem integration, and long-term viability, making it the most robust    
solution for monitoring Kubernetes core resources under large workloads.

Key Arguments:

...

 • Go: Requires familiarity with Kubernetes APIs and Operator SDK workflows, but benefits from robust tooling      
   (e.g., controller-runtime, metrics).

 • Python: Suitable for small-scale or experimental projects but may require custom solutions for advanced         
   Kubernetes features.

 • Rust: A niche option for safety-critical use cases, though its lack of mature operator frameworks may delay     
   deployment.

In conclusion, Go + Operator SDK balances reliability, performance, and ecosystem support, making it the gold      
standard for production-ready Kubernetes observability.

'\n\n**Final Verdict**:  \n\n**Summary of Key Arguments**:  \n- **Go + Operator SDK** (Advocate/Expert): Emphasizes ecosystem maturity, Kubernetes integration, performance, and scalability. The Operator SDK’s robust tooling, active community, and alignment with Kubernetes’ architecture (e.g., CRDs, webhooks) ensure reliability for production-grade operators. Go’s concurrency model and performance benchmarks (10x higher throughput than Python) are critical for monitoring large clusters.  \n- **Python + Kopf** (Critic): Highlights developer productivity, faster prototyping, and simpler syntax. Kopf reduces boilerplate, enabling rapid development, and integrates well with existing observability tools like Prometheus. However, it lacks deep Kubernetes integration and mature tooling for production systems.  \n- **Rust**: Offers safety and performance but lacks a mature Kubernetes operator ecosystem. While appealing for safety-critical systems, its tooling and community are less established 

In [6]:
await create_debate(
    topic="Which is the best fictional universe to live in: Star Wars, Harry Potter, or The Lord of the Rings?",
    context=(
        "A person is choosing a fictional universe to live in permanently. Options include the Star Wars universe (galactic adventures, advanced technology), "
        "the Harry Potter universe (magic, wizarding society), or The Lord of the Rings universe (epic fantasy, medieval-like world). "
        "Consider criteria such as: "
        "1. Quality of life (safety, comfort, opportunities), "
        "2. Cultural and social appeal, "
        "3. Personal freedom and agency, "
        "4. Sense of purpose or adventure, "
        "5. Community and belonging."
    ),
    max_rounds=1,
)

2025-10-12 00:10:00,628 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:10:00,640 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:10:00,653 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:10:00,666 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-12 00:10:00,750 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user wants me to argue that The Lord of the Rings (LotR) is the best fictional universe to live in compared to Star Wars and Harry Potter. Let me start by recalling the key points from the context. The criteria are quality of life, cultural/social appeal, personal freedom, sense of purpose, and community.

First, quality of life. In LotR, the world is more medieval, with less technology. But maybe that's a downside. Wait, but in the books, Middle-earth has a certain stability after the war, though it's still a bit harsh. In contrast, the Harry Potter universe has magic, which could mean more comfort, but there are dangers like dark magic. Star Wars has advanced tech but also the Empire, which might not be safe. So maybe LotR's stability post-war offers a safer environment? But I need to check if that's accurate.

Cultural and social appeal. Both Harry Potter and Star Wars have massive fandoms, but LotR's cultura

Opening Statement:

The Lord of the Rings (LotR) universe is the optimal fictional world to live in, offering unparalleled quality of  
life, profound cultural significance, and a deeply fulfilling sense of purpose. While Star Wars and Harry Potter   
provide compelling worlds, LotR’s blend of stability, epic adventure, and enduring legacy makes it the most        
desirable.

Key Argument 1: Quality of Life

...

Implementation Considerations:

 • Prioritize post-war stability and cultural continuity over short-term conveniences.

 • Acknowledge residual challenges (e.g., lingering evil in LOTR) but weigh them against the absence of systemic   
   oppression seen in Harry Potter or Star Wars.

 • Ultimately, LOTR’s blend of stability, purpose, and timeless themes offers the most holistic and enduring       
   framework for permanent residence.

'\n\n**Final Verdict**  \nThe debate centers on balancing stability, cultural depth, and personal agency. **The Advocate** for *The Lord of the Rings* (LOTR) emphasizes post-war stability, communal harmony, and a universal quest for good, framing Middle-earth as a self-sustaining, low-conflict society with minimal technological intrusion. **The Critic** argues all universes have systemic flaws—Star Wars’ authoritarianism, Harry Potter’s prejudice, and LOTR’s lingering evil—making none ideal for permanent residence. **The Expert** champions Harry Potter’s “magical convenience,” highlighting its structured society, educational systems, and inclusive community as practical advantages.  \n\n**Strongest Points**:  \n- **LOTR’s post-war stability** and cultural resonance (e.g., Shire’s peace, universal themes of sacrifice) offer a timeless, purpose-driven existence.  \n- **Harry Potter’s structured safety** and communal vibrancy provide immediate comfort but may lack long-term cultural depth